# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [2]:
wine_data = load_wine()
x_train, x_test, y_train, y_test = train_test_split(wine_data.data[:, [9, 11, 12]], wine_data.target, 
                                                    random_state=17)

**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [3]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [7]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [23]:
lr = LogisticRegression(random_state=17, multi_class='ovr', solver='lbfgs')
cvs = cross_val_score(lr, X=x_train, y=y_train, cv=skf, )
print('Predicted accuracy:', cvs.mean())
lr.fit(x_train, y_train)
print('accuracy_score:',
      accuracy_score(y_test, lr.predict(x_test)) )

Predicted accuracy: 0.9244810744810745
accuracy_score: 0.9111111111111111


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [24]:
from sklearn.svm import SVC

In [29]:
svc = SVC(random_state=17, gamma='scale')
cvs = cross_val_score(svc, X=x_train, y=y_train, cv=skf, )
print('Predicted accuracy:', cvs.mean())
svc.fit(x_train, y_train)
print('accuracy_score:',
      accuracy_score(y_test, svc.predict(x_test)) )

Predicted accuracy: 0.6921448921448923
accuracy_score: 0.6222222222222222


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [30]:
from sklearn.metrics import classification_report, confusion_matrix

In [31]:
print('LogisticRegression, confusion_matrix:')
print(confusion_matrix(y_test, lr.predict(x_test)) )

print('\nSVC, confusion_matrix:')
print(confusion_matrix(y_test, svc.predict(x_test)) )



LogisticRegression, confusion_matrix:
[[ 9  0  0]
 [ 0 19  0]
 [ 0  4 13]]

SVC, confusion_matrix:
[[ 9  0  0]
 [ 0 19  0]
 [ 2 15  0]]


Вывод:
Для логической регрессии классификатор принял 4 экземпляра вина третьего типа, как за второй. 
Для SVC результат оказался хуже, Он не смог распознать третий тип вина, и отнес данные к первому и второму типу.

### 5.
Для каждой модели выведите classification report.

In [32]:
print('LogisticRegression, classification_report:')
print(classification_report(y_test, lr.predict(x_test)) )

print('\n\nSVC, classification_report:')
print(classification_report(y_test, svc.predict(x_test)) )

LogisticRegression, classification_report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.83      1.00      0.90        19
           2       1.00      0.76      0.87        17

    accuracy                           0.91        45
   macro avg       0.94      0.92      0.92        45
weighted avg       0.93      0.91      0.91        45



SVC, classification_report:
              precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1       0.56      1.00      0.72        19
           2       0.00      0.00      0.00        17

    accuracy                           0.62        45
   macro avg       0.46      0.67      0.54        45
weighted avg       0.40      0.62      0.48        45



C:\Users\HP\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
